# Reality Mining. Jupyter Notebook implementation


With scipy library realitymining mat file is red and saved in to data variable.
Since mat lab contains a lot of data, reading process take a lot of computer resources!

In [1]:
import scipy.io
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import math
from sklearn.neural_network import BernoulliRBM

data = scipy.io.loadmat("realitymining.mat")['s']

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
affilation = data['my_affil']
data_mat = data['data_mat']

# Restricted Boltzmann machine

### Data preprocessing.

Extract data_mat elements for each subject, categorize subjects to sloan and no sloan and save index of subjects who belongs 'sloan' category.

In [170]:
sloan_list = []
features_list = []
frequency_list = []
observation_days = 7
count = 0
frequency_labels = []

for i in range(len(affilation[0])):
    if len(data_mat[0][i]) > 0  and len(affilation[0][i]) > 0:
        frequency_list += [data_mat[0][i]]
        if affilation[0][i][0][0][0] == 'sloan' or affilation[0][i][0][0][0] == 'sloan_2':
            frequency_labels += [1]
        else:
            frequency_labels += [0]
        if len(data_mat[0][i][0]) >= observation_days:
            features_list += [data_mat[0][i]]
            if affilation[0][i][0][0][0] == 'sloan' or affilation[0][i][0][0][0] == 'sloan_2':
                sloan_list += [count]
            count += 1
print(len(features_list))

69


In [171]:
# 1 – home, 2 – work, 3 – elsewhere, 0 – no signal, 4 – phone is off
home, work, elsewhere, no_signal, phone_off = 0, 0, 0, 0, 0
frequency = []
temp = []
for subject in range(len(frequency_list)):
    for hours in range(24):
        for elements in range(len(frequency_list[subject][hours])):
            if frequency_list[subject][hours][elements] == 1:
                home += 1
            elif frequency_list[subject][hours][elements] == 2:
                work += 1
            elif frequency_list[subject][hours][elements] == 3:
                elsewhere += 1
            elif frequency_list[subject][hours][elements] == 0:
                no_signal += 1
            else:
                phone_off += 1
        temp += [home/len(frequency_list[subject][hours]) if home !=0 else 0, 
                     work/len(frequency_list[subject][hours]) if  work !=0 else 0, 
                     elsewhere/len(frequency_list[subject][hours]) if elsewhere !=0 else 0, 
                     #no_signal/len(frequency_list[subject][hours]) if no_signal !=0 else 0,
                     phone_off/len(frequency_list[subject][hours]) if phone_off !=0 else 0]
        home, work, elsewhere, no_signal, phone_off = 0, 0, 0, 0, 0
    frequency += [temp]
    temp = []
frequency_features = np.array(frequency)

### Exclude NaN values.
Change all NaN values to value 4 in feature_list data.

In [172]:
for subject in range(len(features_list)):
    for hour in range(len(features_list[subject])):
        for element in range(len(features_list[subject][hour])):
            if math.isnan(features_list[subject][hour][element]):
                features_list[subject][hour][element] = 4

### Creating Features vector for training and testing.
<br>
__observation data__ -  value of how many days subject activity is stored. By default we have one week - 7 days.
<br>
__all_places__ - it is a list of all possible places in data_set :

 Value | Explanation
  -------------  | -------------
  0 | No signal
  1 | Home
  2 | Work
  3 | Elsewhere
  4 | Phone is off
<br>
__count__ - Current index of element position in features vector.
<br>
__features__ - 2-rd dimensional numpy N-darray for storing features for RBM training:
<br>
* 1-st dimension - Subject
* 2-nd dimension - Subject activity each hour of the week.



In [173]:
#  0 – no signal, 1 – home, 2 – work, 3 – elsewhere, 4 – phone is off

all_places = [0, 1, 2, 3, 4]
count = 0

features = np.zeros((len(features_list), observation_days * 24 * len(all_places)))

for subject in range(len(features_list)):
    for number in range(len(all_places)):
        for week in range(observation_days):
            for hours in range(24):
                if features_list[subject][hours][week] == all_places[number]:
                    features[subject][count] = 1
                count += 1
    count = 0

In [222]:
rbm_list = []  
sk_rbm = BernoulliRBM(n_components=100, verbose=True, learning_rate=0.1, n_iter=1000)
sk_rbm.fit(features)
rbm_list = sk_rbm.transform(features)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -980.99, time = 0.07s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -892.64, time = 0.06s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -885.36, time = 0.04s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -887.83, time = 0.05s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -735.28, time = 0.04s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -675.63, time = 0.06s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -832.11, time = 0.04s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -697.55, time = 0.04s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -534.46, time = 0.04s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -774.95, time = 0.05s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -598.69, time = 0.07s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -779.48, time = 0.04s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -760.06, time = 0.04s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -513.30, time = 0.04s
[BernoulliRBM] 

[BernoulliRBM] Iteration 120, pseudo-likelihood = -317.21, time = 0.07s
[BernoulliRBM] Iteration 121, pseudo-likelihood = -349.91, time = 0.04s
[BernoulliRBM] Iteration 122, pseudo-likelihood = -324.56, time = 0.04s
[BernoulliRBM] Iteration 123, pseudo-likelihood = -294.97, time = 0.04s
[BernoulliRBM] Iteration 124, pseudo-likelihood = -313.18, time = 0.04s
[BernoulliRBM] Iteration 125, pseudo-likelihood = -263.61, time = 0.04s
[BernoulliRBM] Iteration 126, pseudo-likelihood = -309.90, time = 0.06s
[BernoulliRBM] Iteration 127, pseudo-likelihood = -320.42, time = 0.04s
[BernoulliRBM] Iteration 128, pseudo-likelihood = -252.73, time = 0.04s
[BernoulliRBM] Iteration 129, pseudo-likelihood = -275.62, time = 0.04s
[BernoulliRBM] Iteration 130, pseudo-likelihood = -276.46, time = 0.04s
[BernoulliRBM] Iteration 131, pseudo-likelihood = -319.40, time = 0.04s
[BernoulliRBM] Iteration 132, pseudo-likelihood = -234.53, time = 0.06s
[BernoulliRBM] Iteration 133, pseudo-likelihood = -337.96, time 

[BernoulliRBM] Iteration 236, pseudo-likelihood = -190.93, time = 0.06s
[BernoulliRBM] Iteration 237, pseudo-likelihood = -222.29, time = 0.04s
[BernoulliRBM] Iteration 238, pseudo-likelihood = -295.28, time = 0.04s
[BernoulliRBM] Iteration 239, pseudo-likelihood = -200.70, time = 0.04s
[BernoulliRBM] Iteration 240, pseudo-likelihood = -293.73, time = 0.04s
[BernoulliRBM] Iteration 241, pseudo-likelihood = -238.80, time = 0.04s
[BernoulliRBM] Iteration 242, pseudo-likelihood = -136.30, time = 0.05s
[BernoulliRBM] Iteration 243, pseudo-likelihood = -136.62, time = 0.04s
[BernoulliRBM] Iteration 244, pseudo-likelihood = -177.39, time = 0.04s
[BernoulliRBM] Iteration 245, pseudo-likelihood = -204.67, time = 0.04s
[BernoulliRBM] Iteration 246, pseudo-likelihood = -245.51, time = 0.04s
[BernoulliRBM] Iteration 247, pseudo-likelihood = -237.44, time = 0.04s
[BernoulliRBM] Iteration 248, pseudo-likelihood = -216.63, time = 0.05s
[BernoulliRBM] Iteration 249, pseudo-likelihood = -139.15, time 

[BernoulliRBM] Iteration 351, pseudo-likelihood = -173.23, time = 0.07s
[BernoulliRBM] Iteration 352, pseudo-likelihood = -151.71, time = 0.05s
[BernoulliRBM] Iteration 353, pseudo-likelihood = -156.29, time = 0.05s
[BernoulliRBM] Iteration 354, pseudo-likelihood = -137.38, time = 0.05s
[BernoulliRBM] Iteration 355, pseudo-likelihood = -133.62, time = 0.07s
[BernoulliRBM] Iteration 356, pseudo-likelihood = -162.89, time = 0.05s
[BernoulliRBM] Iteration 357, pseudo-likelihood = -101.34, time = 0.08s
[BernoulliRBM] Iteration 358, pseudo-likelihood = -149.51, time = 0.07s
[BernoulliRBM] Iteration 359, pseudo-likelihood = -167.64, time = 0.07s
[BernoulliRBM] Iteration 360, pseudo-likelihood = -64.08, time = 0.05s
[BernoulliRBM] Iteration 361, pseudo-likelihood = -177.29, time = 0.05s
[BernoulliRBM] Iteration 362, pseudo-likelihood = -155.91, time = 0.06s
[BernoulliRBM] Iteration 363, pseudo-likelihood = -178.34, time = 0.09s
[BernoulliRBM] Iteration 364, pseudo-likelihood = -78.78, time = 

[BernoulliRBM] Iteration 466, pseudo-likelihood = -60.52, time = 0.06s
[BernoulliRBM] Iteration 467, pseudo-likelihood = -90.10, time = 0.04s
[BernoulliRBM] Iteration 468, pseudo-likelihood = -145.73, time = 0.03s
[BernoulliRBM] Iteration 469, pseudo-likelihood = -144.02, time = 0.03s
[BernoulliRBM] Iteration 470, pseudo-likelihood = -102.51, time = 0.03s
[BernoulliRBM] Iteration 471, pseudo-likelihood = -171.86, time = 0.03s
[BernoulliRBM] Iteration 472, pseudo-likelihood = -221.79, time = 0.05s
[BernoulliRBM] Iteration 473, pseudo-likelihood = -156.81, time = 0.04s
[BernoulliRBM] Iteration 474, pseudo-likelihood = -64.05, time = 0.03s
[BernoulliRBM] Iteration 475, pseudo-likelihood = -92.75, time = 0.03s
[BernoulliRBM] Iteration 476, pseudo-likelihood = -124.48, time = 0.03s
[BernoulliRBM] Iteration 477, pseudo-likelihood = -69.43, time = 0.03s
[BernoulliRBM] Iteration 478, pseudo-likelihood = -135.39, time = 0.05s
[BernoulliRBM] Iteration 479, pseudo-likelihood = -324.41, time = 0.0

[BernoulliRBM] Iteration 586, pseudo-likelihood = -68.59, time = 0.05s
[BernoulliRBM] Iteration 587, pseudo-likelihood = -175.52, time = 0.04s
[BernoulliRBM] Iteration 588, pseudo-likelihood = -122.87, time = 0.03s
[BernoulliRBM] Iteration 589, pseudo-likelihood = -69.31, time = 0.03s
[BernoulliRBM] Iteration 590, pseudo-likelihood = -71.90, time = 0.03s
[BernoulliRBM] Iteration 591, pseudo-likelihood = -109.62, time = 0.03s
[BernoulliRBM] Iteration 592, pseudo-likelihood = -100.19, time = 0.05s
[BernoulliRBM] Iteration 593, pseudo-likelihood = -124.16, time = 0.04s
[BernoulliRBM] Iteration 594, pseudo-likelihood = -111.66, time = 0.03s
[BernoulliRBM] Iteration 595, pseudo-likelihood = -56.01, time = 0.03s
[BernoulliRBM] Iteration 596, pseudo-likelihood = -70.62, time = 0.03s
[BernoulliRBM] Iteration 597, pseudo-likelihood = -144.35, time = 0.04s
[BernoulliRBM] Iteration 598, pseudo-likelihood = -162.21, time = 0.05s
[BernoulliRBM] Iteration 599, pseudo-likelihood = -187.02, time = 0.0

[BernoulliRBM] Iteration 705, pseudo-likelihood = -123.48, time = 0.09s
[BernoulliRBM] Iteration 706, pseudo-likelihood = -63.72, time = 0.09s
[BernoulliRBM] Iteration 707, pseudo-likelihood = -53.42, time = 0.06s
[BernoulliRBM] Iteration 708, pseudo-likelihood = -233.65, time = 0.07s
[BernoulliRBM] Iteration 709, pseudo-likelihood = -123.61, time = 0.06s
[BernoulliRBM] Iteration 710, pseudo-likelihood = -59.83, time = 0.05s
[BernoulliRBM] Iteration 711, pseudo-likelihood = -71.18, time = 0.06s
[BernoulliRBM] Iteration 712, pseudo-likelihood = -64.19, time = 0.09s
[BernoulliRBM] Iteration 713, pseudo-likelihood = -103.68, time = 0.06s
[BernoulliRBM] Iteration 714, pseudo-likelihood = -114.99, time = 0.05s
[BernoulliRBM] Iteration 715, pseudo-likelihood = -108.09, time = 0.05s
[BernoulliRBM] Iteration 716, pseudo-likelihood = -113.36, time = 0.05s
[BernoulliRBM] Iteration 717, pseudo-likelihood = -95.71, time = 0.05s
[BernoulliRBM] Iteration 718, pseudo-likelihood = -100.46, time = 0.09

[BernoulliRBM] Iteration 821, pseudo-likelihood = -44.88, time = 0.08s
[BernoulliRBM] Iteration 822, pseudo-likelihood = -157.27, time = 0.08s
[BernoulliRBM] Iteration 823, pseudo-likelihood = -96.44, time = 0.06s
[BernoulliRBM] Iteration 824, pseudo-likelihood = -142.37, time = 0.05s
[BernoulliRBM] Iteration 825, pseudo-likelihood = -228.22, time = 0.09s
[BernoulliRBM] Iteration 826, pseudo-likelihood = -174.40, time = 0.09s
[BernoulliRBM] Iteration 827, pseudo-likelihood = -140.90, time = 0.09s
[BernoulliRBM] Iteration 828, pseudo-likelihood = -66.97, time = 0.05s
[BernoulliRBM] Iteration 829, pseudo-likelihood = -137.00, time = 0.07s
[BernoulliRBM] Iteration 830, pseudo-likelihood = -44.67, time = 0.08s
[BernoulliRBM] Iteration 831, pseudo-likelihood = -181.78, time = 0.04s
[BernoulliRBM] Iteration 832, pseudo-likelihood = -42.48, time = 0.04s
[BernoulliRBM] Iteration 833, pseudo-likelihood = -27.61, time = 0.05s
[BernoulliRBM] Iteration 834, pseudo-likelihood = -99.67, time = 0.04s

[BernoulliRBM] Iteration 939, pseudo-likelihood = -59.84, time = 0.05s
[BernoulliRBM] Iteration 940, pseudo-likelihood = -43.95, time = 0.04s
[BernoulliRBM] Iteration 941, pseudo-likelihood = -46.41, time = 0.05s
[BernoulliRBM] Iteration 942, pseudo-likelihood = -151.40, time = 0.04s
[BernoulliRBM] Iteration 943, pseudo-likelihood = -67.35, time = 0.05s
[BernoulliRBM] Iteration 944, pseudo-likelihood = -114.23, time = 0.05s
[BernoulliRBM] Iteration 945, pseudo-likelihood = -94.88, time = 0.05s
[BernoulliRBM] Iteration 946, pseudo-likelihood = -158.36, time = 0.04s
[BernoulliRBM] Iteration 947, pseudo-likelihood = -30.64, time = 0.04s
[BernoulliRBM] Iteration 948, pseudo-likelihood = -53.91, time = 0.04s
[BernoulliRBM] Iteration 949, pseudo-likelihood = -26.96, time = 0.04s
[BernoulliRBM] Iteration 950, pseudo-likelihood = -150.61, time = 0.04s
[BernoulliRBM] Iteration 951, pseudo-likelihood = -40.77, time = 0.04s
[BernoulliRBM] Iteration 952, pseudo-likelihood = -66.83, time = 0.05s
[B

# Multi-layer perceptron with tensorflow

### Labels creation for Multi-layer perceptron
Each subject is classified to sloan or no_sloan.
<br>
If subject belongs to __sloan__ cateogy, his label value is equal __1__.
<br>
If subject belongs to __no sloan__ category, his label value is equal __0__.

In [223]:
labels = []
for i in range(len(features)):
    if i in sloan_list:
        labels += [1]
    else:
        labels += [0]
        
rbm_labels = np.array(labels)

In [224]:
X_train, X_test, y_train, y_test = train_test_split(rbm_list, rbm_labels.ravel(), test_size=0.5, random_state=0)
# Parameters that define the MLP
n_inputs = len(X_train[0])
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 2
X = tf.placeholder(tf.float32, shape= (None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [225]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        # Number of inputs
        n_inputs = int(X.get_shape()[1])
        # This value is computed to randomly initialize the weights
        stddev = 2 / np.sqrt(n_inputs)
        # Weigths can be initialized in different ways
        # Here they are randomly initialized from a Normal distribution (mean=0,std as computed before)
        # Notice that weights are organized in a matrix (tensor) and its number is n_inputs*n_neurons
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        # The variable that will contain the weights is W
        W = tf.Variable(init, name="kernel")
        
        # The variable that will contain the bias is b  
        # and is initialized to zero
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        
        # As in the perceptron what the neurons do is multiply the weights by 
        # the input

        Z = tf.matmul(X, W) + b
        
        # What the activation function does is to "process" the result
        # of the multiplication of weights by inputs, and this is the output
        # of every neuron. 
    
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [226]:
# The scope name for this MLP is "dnn"
with tf.name_scope("dnn"):
    
    # The first hidden layer is defined using the RELU activation function
    # It will contain n_hidden1=300 hidden neurons and therefore output
    # 300 values    
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)

    # The second hidden layer is also defined using the RELU activation function
    # It will contain n_hidden2=100 hidden neurons and therefore output
    # 100 values    
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    
    # The output layer does not use any activation function
    # it will output n_outputs=10 values since there are 10 classes in MNIST
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [227]:
# scope name of the loss function is "loss"
with tf.name_scope("loss"):
    
    # The Loss function is defined. It outputs a loss value for each x
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
#     xentropy = tf.contrib.keras.backend.binary_crossentropy(output=n_inputs, target = n_inputs, from_logits=False)
    
    # The total loss is mean of the loss values 
    loss = tf.reduce_mean(xentropy, name="loss")

In [228]:
learning_rate = 0.01

with tf.name_scope("train"):
    # The plain GradientDescentOptimizer is chosen
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [229]:
with tf.name_scope("eval"):
    # A prediction is correct if it is among the k=1 most probable
    # classes predicted by the NN. Since k=1, it is only correct
    # if the prediction coincides with the true class.
    correct = tf.nn.in_top_k(logits, y, 1)
    
    # The accuracy is the mean of correct predictions
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [230]:
# Initialization of the computation graph
init = tf.global_variables_initializer()

# tensorflow allows to define a saver to store the model after learning
saver = tf.train.Saver()

# Number of epochs
n_epochs = 100

# Size of the batch used to update the gradient
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(70 // batch_size):
            
            #Function next_batch automatically select the batch
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
            
            # Weights are learned using the current batch            
            sess.run(training_op, feed_dict={X: X_train, y: y_train})
            
        # Accuracies are computed in the training and validation sets    
        acc_train = accuracy.eval(feed_dict={X:  X_train, y: y_train})
        acc_val = accuracy.eval(feed_dict={X: X_test,
                                            y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

0 Train accuracy: 0.617647 Val accuracy: 0.571429
1 Train accuracy: 0.647059 Val accuracy: 0.6
2 Train accuracy: 0.676471 Val accuracy: 0.628571
3 Train accuracy: 0.705882 Val accuracy: 0.657143
4 Train accuracy: 0.705882 Val accuracy: 0.657143
5 Train accuracy: 0.705882 Val accuracy: 0.657143
6 Train accuracy: 0.735294 Val accuracy: 0.657143
7 Train accuracy: 0.735294 Val accuracy: 0.657143
8 Train accuracy: 0.735294 Val accuracy: 0.657143
9 Train accuracy: 0.735294 Val accuracy: 0.657143
10 Train accuracy: 0.764706 Val accuracy: 0.657143
11 Train accuracy: 0.794118 Val accuracy: 0.657143
12 Train accuracy: 0.794118 Val accuracy: 0.657143
13 Train accuracy: 0.823529 Val accuracy: 0.657143
14 Train accuracy: 0.823529 Val accuracy: 0.657143
15 Train accuracy: 0.823529 Val accuracy: 0.685714
16 Train accuracy: 0.823529 Val accuracy: 0.714286
17 Train accuracy: 0.823529 Val accuracy: 0.714286
18 Train accuracy: 0.823529 Val accuracy: 0.714286
19 Train accuracy: 0.823529 Val accuracy: 0.71

In [192]:
print(frequency_labels)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0]


In [193]:
#Classificaton from RBM feature
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(frequency_features, frequency_labels, test_size=0.5, random_state=0)
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

model = clf.fit(X_train, y_train) 
predicted_labels = model.predict(X_test)
print("Accuracy %f" % metrics.accuracy_score(y_test, predicted_labels))

Accuracy 0.828571
